
# 🔍 Khám phá dữ liệu MIMIC‑IV (đọc nhẹ, không giải nén toàn bộ)
Notebook này giúp bạn **xem nhanh** và **duyệt theo từng phần** các file `.csv.gz` cực lớn của MIMIC‑IV / MIMIC‑IV‑Note **mà không cần giải nén**, để tránh đầy bộ nhớ.

**Bạn có thể:**
- Xem vài dòng đầu (`nrows`) của từng bảng.
- Đọc theo **chunk** nhỏ (từng lô 5k–50k dòng) để kiểm tra dữ liệu.
- Lấy **mẫu ngẫu nhiên** một số dòng từ file cực lớn (bằng *reservoir sampling* trên các chunk).

> 💡 Phù hợp cho MacBook Air M2 / máy RAM thấp.



## 0) Cấu hình đường dẫn dữ liệu
Điền đúng thư mục gốc chứa 2 cây thư mục sau (theo bạn cung cấp):

```
data/
├── mimic-iv-note/2.2/note/*.csv(.gz)
└── mimiciv/3.1/{hosp,icu}/*.csv.gz
```


In [4]:
from pathlib import Path

# Lùi một cấp ra khỏi notebooks/, rồi vào data/
DATA_ROOT = Path("..") / "data"

HOSP_DIR = DATA_ROOT / "mimiciv" / "3.1" / "hosp"
ICU_DIR  = DATA_ROOT / "mimiciv" / "3.1" / "icu"
NOTE_DIR = DATA_ROOT / "mimic-iv-note" / "2.2" / "note"

for p in [HOSP_DIR, ICU_DIR, NOTE_DIR]:
    print(f"Tồn tại: {p.resolve()} ->", p.exists())


Tồn tại: /Users/lehoangkhang/Tài liệu/revita-sympdiag/data/mimiciv/3.1/hosp -> True
Tồn tại: /Users/lehoangkhang/Tài liệu/revita-sympdiag/data/mimiciv/3.1/icu -> True
Tồn tại: /Users/lehoangkhang/Tài liệu/revita-sympdiag/data/mimic-iv-note/2.2/note -> True



## 1) Liệt kê nhanh các file `.csv.gz`


In [5]:

import glob

def list_gz(dir_path):
    files = sorted(glob.glob(str(dir_path / "*.csv.gz")))
    for f in files:
        print("•", Path(f).name)
    return files

print("📂 HOSP:")
_ = list_gz(HOSP_DIR)
print("\n📂 ICU:")
_ = list_gz(ICU_DIR)
print("\n📂 NOTE:")
_ = list_gz(NOTE_DIR)


📂 HOSP:
• admissions.csv.gz
• d_hcpcs.csv.gz
• d_icd_diagnoses.csv.gz
• d_icd_procedures.csv.gz
• d_labitems.csv.gz
• diagnoses_icd.csv.gz
• drgcodes.csv.gz
• emar.csv.gz
• emar_detail.csv.gz
• hcpcsevents.csv.gz
• labevents.csv.gz
• microbiologyevents.csv.gz
• omr.csv.gz
• patients.csv.gz
• pharmacy.csv.gz
• poe.csv.gz
• poe_detail.csv.gz
• prescriptions.csv.gz
• procedures_icd.csv.gz
• provider.csv.gz
• services.csv.gz
• transfers.csv.gz

📂 ICU:
• caregiver.csv.gz
• chartevents.csv.gz
• d_items.csv.gz
• datetimeevents.csv.gz
• icustays.csv.gz
• ingredientevents.csv.gz
• inputevents.csv.gz
• outputevents.csv.gz
• procedureevents.csv.gz

📂 NOTE:
• discharge.csv.gz
• discharge_detail.csv.gz
• radiology.csv.gz
• radiology_detail.csv.gz



## 2) Hàm tiện ích đọc **nhẹ** (không giải nén toàn bộ)
- `preview_csv(path, nrows)`: xem **n dòng đầu** trực tiếp từ `.csv.gz`  
- `read_in_chunks(path, chunksize)`: duyệt **từng chunk** để kiểm tra cấu trúc/giá trị  
- `show_columns(path)`: chỉ đọc **1 dòng** để lấy **tên cột**
- `sample_rows(path, k, chunksize)`: **lấy mẫu ngẫu nhiên** `k` dòng từ file rất lớn (reservoir sampling)


In [7]:

import pandas as pd
import random

def preview_csv(path, nrows=5):
    df = pd.read_csv(path, compression="gzip", nrows=nrows, low_memory=False)
    print(f"📄 {path} — hiển thị {nrows} dòng đầu:")
    display(df)
    return df

def read_in_chunks(path, chunksize=10000, max_chunks=2, head_each=3):
    i = 0
    for chunk in pd.read_csv(path, compression="gzip", chunksize=chunksize, low_memory=False):
        print(f"🔹 Chunk {i} (khoảng {len(chunk)} dòng) — xem {head_each} dòng đầu:")
        display(chunk.head(head_each))
        i += 1
        if i >= max_chunks:
            break

def show_columns(path):
    df = pd.read_csv(path, compression="gzip", nrows=1, low_memory=False)
    print(f"📑 Cột trong {path}:")
    print(list(df.columns))
    return df.columns.tolist()

def sample_rows(path, k=1000, chunksize=50000, random_state=42):
    """Reservoir sampling cho file cực lớn: chọn ngẫu nhiên k dòng mà không cần load toàn bộ."""
    random.seed(random_state)
    reservoir = None
    total = 0
    for chunk in pd.read_csv(path, compression="gzip", chunksize=chunksize, low_memory=False):
        # lấy ngẫu nhiên các index trong chunk để thêm vào reservoir
        if reservoir is None:
            reservoir = chunk.sample(min(k, len(chunk)), random_state=random_state)
        else:
            m = len(chunk)
            take = min(k, m)
            # với mỗi hàng của chunk, xác suất giữ lại ~ k/(total + i)
            # để đơn giản & hiệu quả, dùng sampling theo tỷ lệ
            add = chunk.sample(take, random_state=random_state)
            combined = pd.concat([reservoir, add], ignore_index=True)
            reservoir = combined.sample(min(k, len(combined)), random_state=random_state)
        total += len(chunk)
    print(f"✅ Lấy mẫu {len(reservoir)} dòng ngẫu nhiên từ tổng ~{total} dòng.")
    return reservoir



## 3) Ví dụ nhanh trên HOSP (admissions, patients, diagnoses_icd, labevents, d_labitems)
> Bạn có thể chạy từng cell dưới đây tuỳ nhu cầu.


In [8]:

admissions_path = HOSP_DIR / "admissions.csv.gz"
patients_path   = HOSP_DIR / "patients.csv.gz"
diagnoses_path  = HOSP_DIR / "diagnoses_icd.csv.gz"
labevents_path  = HOSP_DIR / "labevents.csv.gz"
dlab_path       = HOSP_DIR / "d_labitems.csv.gz"

preview_csv(admissions_path, nrows=5)
show_columns(admissions_path)


📄 ../data/mimiciv/3.1/hosp/admissions.csv.gz — hiển thị 5 dòng đầu:


,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,URGENT,P49AFC,TRANSFER FROM HOSPITAL,HOME,Medicaid,English,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,NaN,EW EMER.,P784FA,EMERGENCY ROOM,HOME,Medicaid,English,WIDOWED,WHITE,2180-06-26 15:54:00,2180-06-26 21:31:00,0
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,NaN,EW EMER.,P19UTS,EMERGENCY ROOM,HOSPICE,Medicaid,English,WIDOWED,WHITE,2180-08-05 20:58:00,2180-08-06 01:44:00,0
3,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,NaN,EW EMER.,P06OTX,EMERGENCY ROOM,HOME,Medicaid,English,WIDOWED,WHITE,2180-07-23 05:54:00,2180-07-23 14:00:00,0
4,10000068,25022803,2160-03-03 23:16:00,2160-03-04 06:26:00,NaN,EU OBSERVATION,P39NWO,EMERGENCY ROOM,NaN,NaN,English,SINGLE,WHITE,2160-03-03 21:55:00,2160-03-04 06:26:00,0


📑 Cột trong ../data/mimiciv/3.1/hosp/admissions.csv.gz:
['subject_id', 'hadm_id', 'admittime', 'dischtime', 'deathtime', 'admission_type', 'admit_provider_id', 'admission_location', 'discharge_location', 'insurance', 'language', 'marital_status', 'race', 'edregtime', 'edouttime', 'hospital_expire_flag']


['subject_id',
 'hadm_id',
 'admittime',
 'dischtime',
 'deathtime',
 'admission_type',
 'admit_provider_id',
 'admission_location',
 'discharge_location',
 'insurance',
 'language',
 'marital_status',
 'race',
 'edregtime',
 'edouttime',
 'hospital_expire_flag']

In [9]:

preview_csv(patients_path, nrows=5)
show_columns(patients_path)


📄 ../data/mimiciv/3.1/hosp/patients.csv.gz — hiển thị 5 dòng đầu:


,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000032,F,52,2180,2014 - 2016,2180-09-09
1,10000048,F,23,2126,2008 - 2010,NaN
2,10000058,F,33,2168,2020 - 2022,NaN
3,10000068,F,19,2160,2008 - 2010,NaN
4,10000084,M,72,2160,2017 - 2019,2161-02-13


📑 Cột trong ../data/mimiciv/3.1/hosp/patients.csv.gz:
['subject_id', 'gender', 'anchor_age', 'anchor_year', 'anchor_year_group', 'dod']


['subject_id',
 'gender',
 'anchor_age',
 'anchor_year',
 'anchor_year_group',
 'dod']

In [ ]:

preview_csv(diagnoses_path, nrows=5)
show_columns(diagnoses_path)


In [10]:

# ⚠️ labevents rất lớn — chỉ xem 5 dòng + duyệt 1-2 chunk nhỏ
preview_csv(labevents_path, nrows=5)
read_in_chunks(labevents_path, chunksize=50000, max_chunks=1, head_each=5)
show_columns(labevents_path)


📄 ../data/mimiciv/3.1/hosp/labevents.csv.gz — hiển thị 5 dòng đầu:


,labevent_id,subject_id,hadm_id,specimen_id,itemid,order_provider_id,charttime,storetime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,priority,comments
0,1,10000032,NaN,2704548,50931,P69FQC,2180-03-23 11:51:00,2180-03-23 15:56:00,___,95.0,mg/dL,70.0,100.0,NaN,ROUTINE,"IF FASTING, 70-100 NORMAL, >125 PROVISIONAL DI..."
1,2,10000032,NaN,36092842,51071,P69FQC,2180-03-23 11:51:00,2180-03-23 16:00:00,NEG,NaN,NaN,NaN,NaN,NaN,ROUTINE,NaN
2,3,10000032,NaN,36092842,51074,P69FQC,2180-03-23 11:51:00,2180-03-23 16:00:00,NEG,NaN,NaN,NaN,NaN,NaN,ROUTINE,NaN
3,4,10000032,NaN,36092842,51075,P69FQC,2180-03-23 11:51:00,2180-03-23 16:00:00,NEG,NaN,NaN,NaN,NaN,NaN,ROUTINE,BENZODIAZEPINE IMMUNOASSAY SCREEN DOES NOT DET...
4,5,10000032,NaN,36092842,51079,P69FQC,2180-03-23 11:51:00,2180-03-23 16:00:00,NEG,NaN,NaN,NaN,NaN,NaN,ROUTINE,NaN


🔹 Chunk 0 (khoảng 50000 dòng) — xem 5 dòng đầu:


,labevent_id,subject_id,hadm_id,specimen_id,itemid,order_provider_id,charttime,storetime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,priority,comments
0,1,10000032,NaN,2704548,50931,P69FQC,2180-03-23 11:51:00,2180-03-23 15:56:00,___,95.0,mg/dL,70.0,100.0,NaN,ROUTINE,"IF FASTING, 70-100 NORMAL, >125 PROVISIONAL DI..."
1,2,10000032,NaN,36092842,51071,P69FQC,2180-03-23 11:51:00,2180-03-23 16:00:00,NEG,NaN,NaN,NaN,NaN,NaN,ROUTINE,NaN
2,3,10000032,NaN,36092842,51074,P69FQC,2180-03-23 11:51:00,2180-03-23 16:00:00,NEG,NaN,NaN,NaN,NaN,NaN,ROUTINE,NaN
3,4,10000032,NaN,36092842,51075,P69FQC,2180-03-23 11:51:00,2180-03-23 16:00:00,NEG,NaN,NaN,NaN,NaN,NaN,ROUTINE,BENZODIAZEPINE IMMUNOASSAY SCREEN DOES NOT DET...
4,5,10000032,NaN,36092842,51079,P69FQC,2180-03-23 11:51:00,2180-03-23 16:00:00,NEG,NaN,NaN,NaN,NaN,NaN,ROUTINE,NaN


📑 Cột trong ../data/mimiciv/3.1/hosp/labevents.csv.gz:
['labevent_id', 'subject_id', 'hadm_id', 'specimen_id', 'itemid', 'order_provider_id', 'charttime', 'storetime', 'value', 'valuenum', 'valueuom', 'ref_range_lower', 'ref_range_upper', 'flag', 'priority', 'comments']


['labevent_id',
 'subject_id',
 'hadm_id',
 'specimen_id',
 'itemid',
 'order_provider_id',
 'charttime',
 'storetime',
 'value',
 'valuenum',
 'valueuom',
 'ref_range_lower',
 'ref_range_upper',
 'flag',
 'priority',
 'comments']

In [ ]:

preview_csv(dlab_path, nrows=5)
show_columns(dlab_path)



## 4) Ví dụ nhanh trên NOTE (discharge, radiology)


In [11]:

discharge_path = NOTE_DIR / "discharge.csv.gz"
radiology_path = NOTE_DIR / "radiology.csv.gz"

preview_csv(discharge_path, nrows=3)
show_columns(discharge_path)

preview_csv(radiology_path, nrows=3)
show_columns(radiology_path)


📄 ../data/mimic-iv-note/2.2/note/discharge.csv.gz — hiển thị 3 dòng đầu:


,note_id,subject_id,hadm_id,note_type,note_seq,charttime,storetime,text
0,10000032-DS-21,10000032,22595853,DS,21,2180-05-07 00:00:00,2180-05-09 15:26:00,\nName: ___ Unit No: _...
1,10000032-DS-22,10000032,22841357,DS,22,2180-06-27 00:00:00,2180-07-01 10:15:00,\nName: ___ Unit No: _...
2,10000032-DS-23,10000032,29079034,DS,23,2180-07-25 00:00:00,2180-07-25 21:42:00,\nName: ___ Unit No: _...


📑 Cột trong ../data/mimic-iv-note/2.2/note/discharge.csv.gz:
['note_id', 'subject_id', 'hadm_id', 'note_type', 'note_seq', 'charttime', 'storetime', 'text']
📄 ../data/mimic-iv-note/2.2/note/radiology.csv.gz — hiển thị 3 dòng đầu:


,note_id,subject_id,hadm_id,note_type,note_seq,charttime,storetime,text
0,10000032-RR-14,10000032,22595853,RR,14,2180-05-06 21:19:00,2180-05-06 23:32:00,EXAMINATION: CHEST (PA AND LAT)\n\nINDICATION...
1,10000032-RR-15,10000032,22595853,RR,15,2180-05-06 23:00:00,2180-05-06 23:26:00,EXAMINATION: LIVER OR GALLBLADDER US (SINGLE ...
2,10000032-RR-16,10000032,22595853,RR,16,2180-05-07 09:55:00,2180-05-07 11:15:00,"INDICATION: ___ HCV cirrhosis c/b ascites, hi..."


📑 Cột trong ../data/mimic-iv-note/2.2/note/radiology.csv.gz:
['note_id', 'subject_id', 'hadm_id', 'note_type', 'note_seq', 'charttime', 'storetime', 'text']


['note_id',
 'subject_id',
 'hadm_id',
 'note_type',
 'note_seq',
 'charttime',
 'storetime',
 'text']


## 5) Lấy **mẫu ngẫu nhiên** từ file rất lớn (không load toàn bộ)
Ví dụ: lấy 2.000 dòng ngẫu nhiên từ `labevents.csv.gz`, sau đó lưu ra file nhỏ để xem nhanh.


In [ ]:

sample = sample_rows(labevents_path, k=2000, chunksize=100000, random_state=7)
out_path = DATA_ROOT / "sample_labevents_2000rows.csv"
sample.to_csv(out_path, index=False)
print("💾 Đã lưu:", out_path)
display(sample.head(10))



## 6) Mẹo tiết kiệm bộ nhớ
- Dùng `nrows` để xem nhanh vài dòng đầu, **không** giải nén toàn bộ.
- Dùng `chunksize` để **duyệt từng phần**, tránh đầy RAM.
- Chỉ chọn **cột cần thiết** bằng `usecols=[...]` nếu bạn biết sẵn danh sách cột.
- Lưu **mẫu nhỏ** ra `.csv` để thử nghiệm pipeline trước khi xử lý toàn bộ dữ liệu.
- Tránh `.read_csv(..., low_memory=True)` với file nhiều kiểu dữ liệu; dùng `low_memory=False` để pandas đoán kiểu ổn định hơn (ít lỗi).
